In [ ]:
import pybullet as p
import pybullet_data
import time
import numpy as np
from fixed_camera import CameraSensor
from robot_view import CameraController
from object_pose import depth_to_pointcloud, get_object_pose_and_grasp

# ---------------- Setup ----------------
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)

camera = CameraController()
plane = p.loadURDF("plane.urdf")
robot = p.loadURDF("final.urdf", useFixedBase=True, flags=p.URDF_USE_SELF_COLLISION)
camera_sensor = CameraSensor(relative_to=robot)
table = p.loadURDF("waste.urdf", useFixedBase=True, flags=p.URDF_USE_SELF_COLLISION)

urdf_files = [
    "obj4.urdf",
    "obj4.urdf",
    "obj4.urdf",
    
]

base_positions = [
    [-0.6, -1.05, .20],
    [-0.25, -0.95, .26],
    [0.05, -0.95, .25],
]

loaded_objects = []
for path, pos in zip(urdf_files, base_positions):
    loaded_objects.append(p.loadURDF(path, basePosition=pos))

num_joints = p.getNumJoints(robot)
print("Available joints:")
rest_pose = [p.getJointState(robot, j)[0] for j in range(num_joints)] 
joint_names = []
for i in range(num_joints):
    info = p.getJointInfo(robot, i)
    name = info[1].decode('utf-8')
    print(f"Joint {i}: {name}")
    joint_names.append(name)

# main_grip_joint = 5
# mimic_grip_joint = 6
# mimic_multiplier = -1

key_map = {
    p.B3G_RIGHT_ARROW: (0, 1),
    p.B3G_LEFT_ARROW:  (0, -1),
    p.B3G_UP_ARROW: (1, 1),
    p.B3G_DOWN_ARROW: (1, -1),
    ord('1'): (2, 1), ord('2'): (2, -1),
    ord('3'): (3, 1), ord('4'): (3, -1),
    ord('5'): (4, 1), ord('6'): (4, -1),
    # ord('q'): (main_grip_joint, 1), ord('a'): (main_grip_joint, -1),
    ord('q'): (5, 1), ord('a'): (5, -1),
    ord('r'): (6, 1), ord('f'): (6, -1),


}

joint_positions = [0.0 for _ in range(num_joints)]
prev_joint_positions = joint_positions.copy()

# Target from camera (will update when pressing P)
# TARGET_POS = np.array([0.73363584 ,-0.31627622 , 0.4446314 ])

# Collect joint limits
joint_limits = []
for i in range(num_joints):
    info = p.getJointInfo(robot, i)
    lower_limit, upper_limit = info[8], info[9]
    if lower_limit > upper_limit:  # Continuous joints
        lower_limit, upper_limit = -3.14, 3.14
    joint_limits.append((lower_limit, upper_limit))

step = 0.015
debug_ids = []

# Camera update timing
update_interval = 05.0
last_update_time = 0.0

# ---------------- Main Loop ----------------
while True:
    if camera is not None:
        try:
            camera.update()
        except Exception:
            pass

    now = time.time()
    if (now - last_update_time) >= update_interval:
        last_update_time = now
        rgb, depth, seg, view_mat, proj_mat = camera_sensor.get_images()
        pts_world, valid_mask = depth_to_pointcloud(
            depth, view_mat, proj_mat, camera_sensor.width, camera_sensor.height
        )
        for d in debug_ids:
            try:
                p.removeUserDebugItem(d)
            except Exception:
                pass
        # debug_ids = get_object_pose(loaded_objects, seg, pts_world, valid_mask)
        results, debug_ids = get_object_pose_and_grasp(
        loaded_objects, seg, pts_world, valid_mask
        )

# Print returned values
        targets = []
        for obj_id, info in results.items():
            print(f"\n[RESULT] Object {obj_id}:")
            print(f"  Centroid   = {info['centroid']}")
            print(f"  Grasp Pos  = {info['grasp_pos']}")
            print(f"  Grasp Quat = {info['grasp_quat']}")
            print(f"  Strategy   = {info['strategy']}")
            # TARGET_POS = np.array(info['grasp_pos'])
            TARGET_POS = np.array(info['grasp_pos'])
            TARGET_POS[2] += 0.03
            targets.append(TARGET_POS)
    # Keyboard control
    keys = p.getKeyboardEvents()
    for k in keys:
        if k in key_map and keys[k] & p.KEY_IS_DOWN:
            joint_id, direction = key_map[k]
            new_position = joint_positions[joint_id] + direction * step
            lower, upper = joint_limits[joint_id]
            new_position = max(min(new_position, upper), lower)
        

            if new_position != joint_positions[joint_id]:
                joint_positions[joint_id] = new_position
                p.setJointMotorControl2(robot, joint_id, p.POSITION_CONTROL, targetPosition=new_position, force=100)

                # if joint_id == main_grip_joint:
                #     mimic_position = mimic_multiplier * new_position
                #     joint_positions[mimic_grip_joint] = mimic_position
                #     p.setJointMotorControl2(robot, mimic_grip_joint, p.POSITION_CONTROL, targetPosition=mimic_position, force=100)
        

############################################################################################################################
        # num_joints = p.getNumJoints(robot)
        # rest_pose = [p.getJointState(robot, j)[0] for j in range(num_joints)]  # joint angles
###
        if k == ord('p') and keys[k] & p.KEY_WAS_TRIGGERED:
            print(f"[INFO] Moving to target: {TARGET_POS}")
            if len(targets) == 0:
                print("[INFO] No objects detected. Nothing to pick.")
            else:
                for i, TARGET_POS in enumerate(targets, 1):
                    print(f"\n[INFO] Picking Object {i}/{len(targets)}")

            end_effector_index = num_joints - 1  # Last joint (EEF)
            place_pos = np.array([0.73363584, -0.31627622, 0.5846314])  # Fixed place location
            gripper_joint_indices = [num_joints - 2, num_joints -1]  # Example: last 2 joints are gripper fingers
            left_finger = num_joints - 2
            right_finger = num_joints - 1

    # --- Step 1: Move to grasp pose ---
            joint_solution = p.calculateInverseKinematics(robot, end_effector_index, TARGET_POS)
            for idx in range(len(joint_solution)):
                lower, upper = joint_limits[idx]
                clamped_pos = max(min(joint_solution[idx], upper), lower)
                p.setJointMotorControl2(robot, idx, p.POSITION_CONTROL,
                                        targetPosition=clamped_pos, force=200 , maxVelocity=0.15)

            for _ in range(2000):  # Let simulation reach grasp point
                p.stepSimulation()

    # --- Step 2: Close gripper (grasp object) ---
            # for joint in gripper_joint_indices:
            p.setJointMotorControl2(robot, left_finger,  p.POSITION_CONTROL, targetPosition= 1.2, force=200 ,maxVelocity=0.2)
            p.setJointMotorControl2(robot, right_finger, p.POSITION_CONTROL, targetPosition=-1.2, force=200 , maxVelocity=0.2)
                # p.setJointMotorControl2(robot, joint, p.POSITION_CONTROL, targetPosition=1.08, force=200)
            for _ in range(2000):
                p.stepSimulation()

            print("[INFO] Object grasped.")

    # --- Step 3: Move to place position ---
            joint_solution = p.calculateInverseKinematics(robot, end_effector_index, place_pos)
            for idx in range(len(joint_solution)):
                lower, upper = joint_limits[idx]
                clamped_pos = max(min(joint_solution[idx], upper), lower)
                p.setJointMotorControl2(robot, idx, p.POSITION_CONTROL,
                                        targetPosition=clamped_pos, force=200 ,maxVelocity=0.15)

            for _ in range(2000):  # Move to placement location
                p.stepSimulation()

    # --- Step 4: Open gripper (release object) ---
            p.setJointMotorControl2(robot, left_finger,  p.POSITION_CONTROL, targetPosition= -1.2, force=200 ,maxVelocity=0.2)
            p.setJointMotorControl2(robot, right_finger, p.POSITION_CONTROL, targetPosition= 1.2, force=200 ,maxVelocity=0.2)

            # for joint in gripper_joint_indices:
            #     p.setJointMotorControl2(robot, joint, p.POSITION_CONTROL, targetPosition=-1.08, force=200)  
        # adjust 0.04 based on gripper open width
            for _ in range(2000):
                p.stepSimulation()

            print("[INFO] Object placed.")
            ##
            # print("[INFO] Returning to rest pose...")
            for _ in range(2000):   # smooth return
                for j in range(num_joints):
                    p.setJointMotorControl2(robot, j, p.POSITION_CONTROL,
                                        targetPosition=rest_pose[j], force=200, maxVelocity=3)
            p.stepSimulation()
            print("[INFO] Robot returned to rest.")
    #####################################################################################################################
    p.stepSimulation()

    # Track joint movement
    tol = 1e-4
    current_positions = []
    for i in range(num_joints):
        state = p.getJointState(robot, i)
        pos = state[0]
        current_positions.append(pos)

    for i in range(num_joints):
        if abs(current_positions[i] - prev_joint_positions[i]) > tol:
            name = joint_names[i] if i < len(joint_names) else str(i)
            # print(f"Joint {i} ({name}) moved: {prev_joint_positions[i]:.6f} -> {current_positions[i]:.6f}")
    prev_joint_positions = current_positions

    time.sleep(0.01)
